In [1]:
begin
	using Flux
	using Flux: onehot
	using Flux: gradient
	using Flux.Optimise: update!
	using Flux: onecold
	using Transformers
	using Transformers.Basic 
    using Transformers.Datasets: batched
    enable_gpu(false) 
    using NNlib
	using Einsum
	using PyCall
	
	nn = pyimport("flax.linen") #das funktioniert noch nicht; dieses Python-Package muss aber importiert werden, da es nichts äquivalentes zu Julia gibt
end

PyCall.PyError: PyCall.PyError("PyImport_ImportModule\n\nThe Python package flax.linen could not be imported by pyimport. Usually this means\nthat you did not install flax.linen in the Python version being used by PyCall.\n\nPyCall is currently configured to use the Julia-specific Python distribution\ninstalled by the Conda.jl package.  To install the flax.linen module, you can\nuse `pyimport_conda(\"flax.linen\", PKG)`, where PKG is the Anaconda\npackage that contains the module flax.linen, or alternatively you can use the\nConda package directly (via `using Conda` followed by `Conda.add` etcetera).\n\nAlternatively, if you want to use a different Python distribution on your\nsystem, such as a system-wide Python (as opposed to the Julia-specific Python),\nyou can re-configure PyCall with that Python.   As explained in the PyCall\ndocumentation, set ENV[\"PYTHON\"] to the path/name of the python executable\nyou want to use, run Pkg.build(\"PyCall\"), and re-launch Julia.\n\n", PyObject

In [14]:
function elu_feature_map(x)
  return NN.elu(x) + 1
end

elu_feature_map (generic function with 1 method)

In [18]:
function linear_attention(query, key, value, dropout_rng, dropout_rate, broadcast_dropout,deterministic)

    broadcast_dropout=True
    dropout_rng=None
    dropout_rate=0.
    deterministic=False
    feature_map = elu_feature_map()
    eps=1e-6

    Base.delete!(broadcast_dropout)
    Base.delete!(dropout_rng)
    Base.delete!(dropout_rate)
    Base.delete!(deterministic)

   # Base.@assert key.ndim == query.ndim == value.ndim == 4
   # Base.@assert key.shape[:-1] == value.shape[:-1]
   # Base.@assert (query.shape[0:1] == key.shape[0:1] and query.shape[-1] == key.shape[-1])

    query_mapped = feature_map(query)
    key_mapped = feature_map(key)
    
    @einsum kv[n,h,m,d] = key_mapped[n,s,h,d]*value[n,s,h,m]

    sum_1 = sum(key_mapped, axis=1)+eps

    @einsum z_1[n,l,h] = query_mapped[n,l,h,d]*sum_1[n,h,d]

    z = 1/z_1
    
    @einsum y[n,l,h,m] = query_mapped[n,l,h,d]*kv[n,h,m,d]*z[n,l,h]

    return y
end

linear_attention (generic function with 1 method)

In [7]:
function apply(
        inputs_q,
        inputs_kv,
        num_heads,
        dtype,
        qkv_features,
        out_features,
        causal_mask,
        padding_mask,
        key_padding_mask,
        segmentation,
        key_segmentation,
        cache,
        broadcast_dropout,
        dropout_rng,
        dropout_rate,
        deterministic,
        precision,
        kernel_init,
        bias_init,
        bias)

    dtype=jnp.float32,
    qkv_features=None,
    out_features=None,
    causal_mask=False,
    padding_mask=None,
    key_padding_mask=None,
    segmentation=None,
    key_segmentation=None,
    cache=None,
    broadcast_dropout=True,
    dropout_rng=None,
    dropout_rate=0.,
    deterministic=False,
    precision=None,
    kernel_init=nn.linear.default_kernel_init, #diese Funktionen gibt es für Julia nichts
    bias_init=nn.initializers.zeros,            # diese auch nicht
    bias=True

    if padding_mask == nothing
        NotImplementedError(
            'Currently, we do not support autoregresive decoding.')
  
       # Base.@assert causal_mask or not cache, (
            'Caching is only support for causal attention.')
    end

    Base.@assert inputs_q.ndim == 3
        
    if inputs_kv == nothing
       inputs_kv = inputs_q
    end
  
      features = out_features || inputs_q.shape[-1]
      qkv_features = qkv_features || inputs_q.shape[-1]

      #Base.@assert qkv_features % num_heads == 0, (
       # 'Memory dimension must be divisible by number of heads.')
    
    head_dim = qkv_features // num_heads

    dense = nn.DenseGeneral.partial(    #diese funktion gibt es in keinem Julia package -> PyCall wird notwendig sein
        axis=-1,
        features=(num_heads, head_dim),
        kernel_init=kernel_init,
        bias_init=bias_init,
        bias=bias,
        precision=precision)

        query, key, value = (dense(inputs_q, dtype=dtype, name='query'),
        dense(inputs_kv, dtype=dtype, name='key'),
        dense(inputs_kv, dtype=dtype, name='value'))


    # apply regular dot product attention
    x = linear_attention(
            query,
            key,
            value,
            dropout_rng,
            dropout_rate,
            broadcast_dropout,
            deterministic)

    # back to the original inputs dimensions
    out = nn.DenseGeneral(
                x,
                features,
                (-2, -1),
                kernel_init,
                bias_init,
                bias,
                dtype,
                precision,
                'out')

        return out

end

ErrorException: syntax: character literal contains multiple characters

In [ ]:
class LinearTransformerBlock(nn.Module):
  """FastTransformer layer (https://arxiv.org/abs/2006.16236)."""

function apply(self,
            inputs,
            qkv_dim,
            mlp_dim,
            num_heads,
            dtype,
            inputs_segmentation,
            causal_mask,
            padding_mask,
            dropout_rate,
            attention_dropout_rate,
            deterministic,
            cache)

    dtype=jnp.float32,
    inputs_segmentation=None,
    causal_mask=False,
    padding_mask=None,
    dropout_rate=0.1,
    attention_dropout_rate=0.1,
    deterministic=False,
    cache=None
    """Applies LinearTransformer module.
    Args:
      inputs: input data
      qkv_dim: dimension of the query/key/value
      mlp_dim: dimension of the mlp on top of attention block
      num_heads: number of heads
      dtype: the dtype of the computation (default: float32).
      inputs_segmentation: input segmentation info for packed examples.
      causal_mask: bool, mask future or not
      padding_mask: bool, mask padding tokens
      dropout_rate: dropout rate
      attention_dropout_rate: dropout rate for attention weights
      deterministic: bool, deterministic or not (to apply dropout)
      cache: flax autoregressive cache for fast decoding.
    Returns:
      output after transformer block.
    """

    # Attention block.
    assert inputs.ndim == 3
    x = nn.LayerNorm(inputs)
    x = linear_attention.LinearSelfAttention(
        x,
        num_heads=num_heads,
        dtype=dtype,
        qkv_features=qkv_dim,
        causal_mask=causal_mask,
        segmentation=inputs_segmentation,
        padding_mask=padding_mask,
        kernel_init=nn.initializers.xavier_uniform(),
        bias_init=nn.initializers.normal(stddev=1e-6),
        bias=False,
        broadcast_dropout=False,
        dropout_rate=attention_dropout_rate,
        deterministic=deterministic,
        cache=cache)
    x = nn.dropout(x, rate=dropout_rate, deterministic=deterministic)
    x = x + inputs

    # MLP block.
    y = nn.LayerNorm(x)
    y = common_layers.MlpBlock(
        y,
        mlp_dim=mlp_dim,
        dtype=dtype,
        dropout_rate=dropout_rate,
        deterministic=deterministic)

    return x + y


class LinearTransformerEncoder(nn.Module):
  """Linear Transformer Model Encoder."""

  def apply(self,
            inputs,
            vocab_size,
            inputs_positions=None,
            inputs_segmentation=None,
            shared_embedding=None,
            use_bfloat16=False,
            emb_dim=512,
            num_heads=8,
            dtype=jnp.float32,
            num_layers=6,
            qkv_dim=512,
            mlp_dim=2048,
            max_len=512,
            train=True,
            dropout_rate=0.1,
            attention_dropout_rate=0.1,
            learn_pos_emb=False,
            classifier=False,
            classifier_pool='CLS',
            num_classes=10):
    """Applies Transformer model on the inputs.
    Args:
      inputs: input data
      vocab_size: size of the vocabulary
      inputs_positions: input subsequence positions for packed examples.
      inputs_segmentation: input segmentation info for packed examples.
      shared_embedding: a shared embedding layer to use.
      use_bfloat16: bool: whether use bfloat16.
      emb_dim: dimension of embedding
      num_heads: number of heads
      dtype: the dtype of the computation (default: float32)
      num_layers: number of layers
      qkv_dim: dimension of the query/key/value
      mlp_dim: dimension of the mlp on top of attention block
      max_len: maximum length.
      train: if it is training,
      dropout_rate: dropout rate
      attention_dropout_rate: dropout rate for attention weights
      learn_pos_emb: boolean, if learn the positional embedding or use the
        sinusoidal positional embedding.
      classifier: boolean, for classification mode (output N-class logits)
      classifier_pool: str, supports "MEAN", "MAX" pooling.
      num_classes: int, number of classification classes.
    Returns:
      output of a transformer encoder or logits if classifier_mode is true.
    """
    assert inputs.ndim == 2  # (batch, len)

    # Padding Masks
    src_padding_mask = (inputs > 0)[..., None]

    # Input Embedding
    if shared_embedding is None:
      input_embed = nn.Embed.partial(
          num_embeddings=vocab_size,
          features=emb_dim,
          embedding_init=nn.initializers.normal(stddev=1.0))
    else:
      input_embed = shared_embedding
    x = inputs.astype('int32')
    x = input_embed(x)

    if classifier and classifier_pool == 'CLS':
      cls = self.param('cls', (1, 1, emb_dim), nn.initializers.zeros)
      cls = jnp.tile(cls, [x.shape[0], 1, 1])
      x = jnp.concatenate([cls, x], axis=1)
      max_len += 1
      src_padding_mask = jnp.concatenate(
          [src_padding_mask[:, :1], src_padding_mask], axis=1)
    pe_init = nn.initializers.normal(stddev=0.02) if learn_pos_emb else None
    x = common_layers.AddPositionEmbs(
        x,
        inputs_positions=inputs_positions,
        posemb_init=pe_init,
        max_len=max_len,
        name='posembed_input')
    x = nn.dropout(x, rate=dropout_rate, deterministic=not train)

    if use_bfloat16:
      x = x.astype(jnp.bfloat16)
      dtype = jnp.bfloat16
    else:
      dtype = jnp.float32

    # Input Encoder
    for lyr in range(num_layers):
      x = LinearTransformerBlock(
          x,
          qkv_dim=qkv_dim,
          mlp_dim=mlp_dim,
          num_heads=num_heads,
          dtype=dtype,
          padding_mask=src_padding_mask,
          inputs_segmentation=inputs_segmentation,
          dropout_rate=dropout_rate,
          attention_dropout_rate=attention_dropout_rate,
          deterministic=not train,
          name=f'encoderblock_{lyr}')
    encoded = nn.LayerNorm(x, dtype=dtype, name='encoder_norm')

    if classifier:
      encoded = common_layers.classifier_head(
          encoded, num_classes, mlp_dim, pooling_mode=classifier_pool)
    return encoded


class LinearTransformerDualEncoder(nn.Module):
  """Linear Transformer Model for Matching (dual encoding) tasks."""

  def apply(self,
            inputs1,
            inputs2,
            vocab_size=None,
            inputs1_positions=None,
            inputs2_positions=None,
            inputs1_segmentation=None,
            inputs2_segmentation=None,
            use_bfloat16=False,
            emb_dim=512,
            num_heads=8,
            num_layers=6,
            qkv_dim=512,
            mlp_dim=2048,
            max_len=2048,
            train=False,
            dropout_rate=0.1,
            attention_dropout_rate=0.1,
            classifier=True,
            classifier_pool='CLS',
            num_classes=2,
            interaction=None):
    """Applies Transformer model on text similarity.
    A deliberate choice to distinguish this from NLI because
    we may want to do different things to the model later. Dual Encoding
    mode enforces that we do not do cross attention between pairs.
    Args:
      inputs1: input data.
      inputs2: target data.
      vocab_size: size of the input vocabulary.
      inputs1_positions: input subsequence positions for packed examples.
      inputs2_positions: target subsequence positions for packed examples.
      inputs1_segmentation: input segmentation info for packed examples.
      inputs2_segmentation: target segmentation info for packed examples.
      use_bfloat16: bool: whether use bfloat16.
      emb_dim: dimension of embedding.
      num_heads: number of heads.
      num_layers: number of layers.
      qkv_dim: dimension of the query/key/value.
      mlp_dim: dimension of the mlp on top of attention block.
      max_len: maximum length.
      train: whether it is training.
      dropout_rate: dropout rate.
      attention_dropout_rate: dropout rate for attention weights.
      classifier: boolean, to use classifier.
      classifier_pool: str, supports "MEAN", "MAX" pooling.
      num_classes: int, number of classification classes.
      interaction: str
    Returns:
      output of a transformer decoder.
    """
    encoder = LinearTransformerEncoder.shared(
        inputs_positions=inputs1_positions,
        inputs_segmentation=inputs1_segmentation,
        vocab_size=vocab_size,
        use_bfloat16=use_bfloat16,
        emb_dim=emb_dim,
        num_heads=num_heads,
        num_layers=num_layers,
        qkv_dim=qkv_dim,
        mlp_dim=mlp_dim,
        max_len=max_len,
        train=train,
        dropout_rate=dropout_rate,
        attention_dropout_rate=attention_dropout_rate,
        name='encoder')
    inputs1_encoded = encoder(inputs1)
    inputs2_encoded = encoder(inputs2)

    encoded = common_layers.classifier_head_dual(
        inputs1_encoded,
        inputs2_encoded,
        num_classes,
        mlp_dim,
        pooling_mode=classifier_pool,
        interaction=interaction)

    return encoded